In [1]:
from data import Corpus
import trainer
from utils import batchify
import json
import tensorflow as tf

In [ ]:
# corpus = Corpus('baomoi/')
!rm -rf 1 && mkdir -p 1/checkpoints

In [4]:
import numpy as np

In [ ]:
# np.save('baomoi/train.npy', corpus.train)

In [ ]:
# np.save('baomoi/valid.npy', corpus.valid)

In [ ]:
# np.save('baomoi/test.npy', corpus.test)

In [ ]:
# with open('word2idx.json','w') as out:
#     json.dump(corpus.dictionary.word2idx, out)

In [2]:
with open('word2idx.json','r') as inp:
    word2idx = json.load(inp)

In [5]:
with open('baomoi/train.npy','rb') as inp:
    train = np.load(inp)
with open('baomoi/test.npy','rb') as inp:
    test = np.load(inp)
with open('baomoi/valid.npy','rb') as inp:
    valid = np.load(inp)

In [ ]:
train_data = batchify(train, 102).T
val_data = batchify(valid, 10).T

In [ ]:
VERSION = 1
params = dict(
    model_configs = {
        'rnn_layers': [
            {'units': 1150, 'input_size': 400, 'drop_i': 0.3, 'drop_w': 0.5},
            {'units': 1150, 'input_size': 1150, 'drop_w': 0.5},
            {'units': 400, 'input_size': 1150, 'drop_o': 0.3, 'drop_w': 0.5}
        ],
        'vocab_size': len(word2idx), 
        'drop_e': 0.1,
        'parallel_iterations': 64
    },
    optimizer = tf.train.GradientDescentOptimizer,
    learning_rate = 10.0,
    decay_freq = 10000,
    decay_rate = 0.1,
    alpha = 1e-5,
    beta = 1e-5,
    clip_norm = 0.25,
    bptt = 100,
    use_ema = True,
    log_path = '1/logs',
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

In [ ]:
my_trainer = trainer.Trainer(**params)

In [ ]:
tf.reset_default_graph()
my_trainer.build()

In [ ]:
my_trainer.session.run(tf.global_variables_initializer())
for _ in range(10000):
    my_trainer.train_dev_loop(train_data, val_data)